In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [ ]:
from sklearn import model_selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [ ]:
data = pd.read_json('songs_metadata.json')

Check the first five rows to get an idea.

In [ ]:
data.head()

We observe a 'spotify' and 'lastfm' in front of some columns. Let's remove that.

In [ ]:
cols = data.columns

In [ ]:
cols

Remove Spotify from column names.

In [ ]:
# this splits when it finds '-' and gets the rest of the words
# if it doesn't find '-' it gets the original word

data.columns = ['_'.join(i.split('-')[1:]) if len(i.split('-')) > 1 else i for i in cols]

Drop duplicate columns.

In [ ]:
data = data.drop(['trackName', 'artistName'], axis=1)

Rename albumName column.

In [ ]:
data = data.rename(columns= {'albumName' : 'album'})

Check for NaN values.

In [ ]:
data.isna().sum()

Visualize some of them.

In [ ]:
data[data.isna().any(axis=1)]

Drop NaN values (we could do other things such as fill them with zero, mean etc).

In [ ]:
# We will use this one to have access to all the variables, no NaN values
ds = data.dropna()

ds.isna().sum()

In [ ]:
# We can keep this dataset if we didn't plan to use the play/listener counts
df = data.dropna(subset = ['popularity'])

df.isna().sum()

Extract the year and the duration in minutes from each song.

In [ ]:
year = ds.loc[:, 'date'].str.split('-').str[0].astype(int)
minutes = ds.loc[:, 'duration_ms']/(1000*60)

In [ ]:
ds = ds.drop('duration_ms', axis = 1)

In [ ]:
ds.loc[:, 'year'] = year

ds.loc[:, 'duration_minutes'] = minutes

Define some functions to apply to several columns.

In [ ]:
def find_tracks(dataset, feature_col, k=10, artist_col=0, song_col=1):
    '''
    Returns the top k (default = 10) tracks with their corresponding artists depending on the music feature of our choice.
    
    :parameter dataset: dataset to analyse.
    :parameter feature_col: # position of column with the feature of choice.
    :parameter k: number of top k tracks to return (default = 10).
    :parameter artist_col: # position of column with artists (zero-indexed) e.g. 1st column -> 0.
    :parameter song_col: # position of column with songs (zero-indexed) e.g. 2nd column -> 1.
    :return: dataframe with the top k tracks together with their artists.
    '''
    
    dataset = ds
    ds_sort = ds.sort_values(by = ds.columns[feature_col], ascending = False).reset_index(drop=True)
    return (ds_sort.iloc[:k, [song_col, artist_col]])

def find_artists(dataset, feature_col, k=5):
    '''
    Returns the top k artists with the most tracks depending on the music feature of our choice (among the first 500).
    
    :parameter dataset: dataset to analyse.
    :parameter feature_col: # position of column with the feature of choice.
    :parameter k: number of top k artists to return (default = 5).
    :return: dataframe with the top k artists together with their song counts and the chosen music feature.
    '''
    
    dataset = ds
    all_artists = find_tracks(ds, feature_col, 500).iloc[:,1].value_counts().rename_axis('Artist').reset_index(name ='Tracks')
    all_artists['Feature'] = ds.columns[feature_col]
    
    return all_artists.iloc[:k, :]

Let's loop for each music characheristic and print the first 10 tracks with that feature.

In [ ]:
names = ds.columns.values.tolist()

In [ ]:
for i in range(8, 19):
    print(f"\nThe top 10 tracks for {names[i]} are:\n\n{find_tracks(ds, i)}.")

We can use the second column of our output to count how many tracks with that feature each artist has.

In [ ]:
# example for danceability

# gets the 2nd column for the top 500 tracks and counts the number of times each artist has appeared 
# finally it outputs the 5 artists with the most tracks with that characteristic


find_tracks(ds, 8, 500).iloc[:,1].value_counts()[:5]

That's what our find_artists function does (and also presents the music characteristic).

In [ ]:
find_artists(ds, 8)

Now we will use the defined function to search for 5 artists with the most tracks (among the first 500) with that feature.

In [ ]:
for i in range(8, 19):
    print(f"\nThe top 5 artists for {names[i]} are:\n\n{find_artists(ds, i).iloc[:, :2]}.")

Plot these results for 4 music characteristics.

In [ ]:
dance = find_artists(ds, 8)
energy = find_artists(ds, 9)
loud = find_artists(ds, 11)
acoustic = find_artists(ds, 14)

top_artists = pd.concat([dance, energy, loud, acoustic])

In [ ]:
fig = plt.figure(figsize = (28, 25))


sns.barplot(x = top_artists['Artist'], y = top_artists['Tracks'], hue = top_artists['Feature'])
plt.legend(fontsize = 21)
plt.title('Artists with their characteristics', fontsize = 25)
plt.xlabel('Artists', fontsize = 20)
plt.ylabel('Number of songs', fontsize = 20)
plt.xticks(fontsize = 17, rotation = 20)
plt.yticks(fontsize = 17)

plt.show()

Now let's count the total songs for each artist.

The code below aggregates the number of tracks for each artist and takes the first 20 artists with the most songs.

In [ ]:
aggr = ds.groupby('artist')['track'].agg(len).sort_values(ascending = False)[:20]

aggr

In [ ]:
fig = plt.figure(figsize = (30, 30))

sns.barplot(x = aggr.index.values, y = aggr, palette="Blues_d")

plt.xlabel('Artist Name', fontsize = 23)
plt.ylabel('Count of songs', fontsize = 23)
plt.xticks(fontsize = 20, rotation = 30)
plt.yticks(fontsize = 20)
plt.title('Artist Name vs Count of songs', fontsize = 30)

plt.show()

Plot histograms for every variable.

In [ ]:
fig = ds.hist(figsize=(30, 30), xlabelsize = 15, ylabelsize = 15)

[x.title.set_size(20) for x in fig.ravel()]

plt.show()

Plot positive correlations between features.

In [ ]:
f, ax = plt.subplots(figsize = (20, 15))

ax = sns.heatmap(ds.corr(), vmin = 0, vmax = 1, annot = True)

plt.autoscale()

We observe that:

- Valence is correlated with danceability and energy.
- Loudness is correlated with energy.
- Number of tracks in albums is correlated with track number (why?).

Plot negative correlations between features.

In [ ]:
f, ax = plt.subplots(figsize = (20, 15))

ax = sns.heatmap(ds.corr(), vmin = -1, vmax = 0, annot = True)

plt.autoscale()

We observe that:

- Popularity is negatively correlated with acousticness (and less with instrumentalness).
- Acousticness is negatively correlated with energy, danceability, loudness, duration in minutes (and popularity).

Why is track number and number of tracks correlated?

In [ ]:
# partial answer

# number of instances that have 1 track in album (and track number = 1) -> singles

print(len(data[(data['track_no']==1.0) & (data['num_of_tracks_in_album']==1.0)]))

Let's plot a scatterplot/regplot for the biggest correlations e.g. loudness-energy, valence-danceability, acousticness-energy.

In [ ]:
# Loudness vs Energy (correlation  = 0.74)

plt.figure(figsize=(18, 15))

sns.regplot(x = ds['loudness'], y = ds['energy'])

plt.xlabel('Loudness', fontsize = 14)
plt.ylabel('Energy', fontsize = 14)
plt.xticks(fontsize = 11)
plt.yticks(fontsize = 11)
plt.title('Loudness vs Energy', fontsize = 18)

plt.show()

In [ ]:
# Valence vs Danceability (correlation  = 0.54)

plt.figure(figsize=(18, 15))

sns.regplot(x = ds['valence'], y = ds['danceability'])

plt.xlabel('Valence', fontsize = 14)
plt.ylabel('Danceability', fontsize = 14)
plt.xticks(fontsize = 11)
plt.yticks(fontsize = 11)
plt.title('Valence vs Danceability', fontsize = 18)

plt.show()

In [ ]:
# Acousticness vs Energy (correlation = - 0.7)

plt.figure(figsize=(18, 15))

sns.regplot(x = ds['acousticness'], y = ds['energy'])

plt.xlabel('Acousticness', fontsize = 14)
plt.ylabel('Energy', fontsize = 14)
plt.xticks(fontsize = 11)
plt.yticks(fontsize = 11)
plt.title('Acousticness vs Energy', fontsize = 18)

plt.show()

Let's see how these characteristics have changed over the years

In [ ]:
plt.figure(figsize=(18, 15))
sns.set()

columns = ["acousticness","danceability","energy","speechiness","liveness","valence"]
for col in columns:
    x = ds.groupby("year")[col].mean()
    ax= sns.lineplot(x=x.index, y=x, label=col)

ax.set_title('Audio characteristics over year', fontsize = 16)
ax.set_ylabel('Measure', fontsize = 14)
ax.set_xlabel('Year', fontsize = 14)
plt.legend(fontsize= 14)
plt.xlim(1940, 2020)

plt.show()

Define a feature set and try to predict our target variable

In [ ]:
X = ds.iloc[:, np.r_[8:19, 28]]

Y = ds['popularity']

According to the previous correlations we have an idea for the characteristics that make music popular e.g. energy/loudness vs acousticness/instrumentalness.

Let's fit a Random Forest Regressor to extract the most important features when predicting track popularity.

In [ ]:
forest = RandomForestRegressor(n_estimators = 100, random_state = 42)
forest.fit(X, Y)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print(f"{f+1}. feature {indices[f]}: {X.columns[indices[f]]} ({importances[indices[f]]})")

# Plot the feature importances of the forest
plt.figure(figsize = (30,20))
plt.title("Feature importances", fontsize = 25)
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr = std[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], fontsize = 18, rotation = 20)
plt.yticks(fontsize = 18)
plt.xlim([-1, X.shape[1]])
plt.show()

We will use the first 6 important features from Random Forest and danceability from our correlation analysis.

In [ ]:
X_new = X.iloc[:, np.r_[6, 11, 7, 3, 8, 1, 0]]

In [ ]:
quantile_list = [0.0, 0.50, 1.0]
quantiles = ds['popularity'].quantile(quantile_list)

labels = [0, 1]
ds['labels'] = pd.cut(ds['popularity'], quantiles, labels=labels, include_lowest = True)

In [ ]:
Y_new  = ds['labels']

Evaluate common classifiers with 10-fold cross-validation.

In [ ]:
models = []
models.append(('LR', LogisticRegression(solver = 'lbfgs', max_iter=500)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC(gamma = 'auto')))
models.append(('Random Forest', RandomForestClassifier(n_estimators= 100, random_state = 42)))

# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state = 42)
    cv_results = model_selection.cross_val_score(model, X_new, Y_new, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print(f"The mean accuracy of {name} is {cv_results.mean()} with standard deviation {cv_results.std()}")